In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ai-w3p2/train_data.csv
/kaggle/input/2022-ai-w3p2/test_data.csv
/kaggle/input/2022-ai-w3p2/sample_submit.csv


In [2]:
train_data = pd.read_csv('/kaggle/input/2022-ai-w3p2/train_data.csv')
test_data = pd.read_csv('/kaggle/input/2022-ai-w3p2/test_data.csv')

In [3]:
from sklearn.preprocessing import LabelEncoder
cls = LabelEncoder()
train_data['model'] = cls.fit_transform(train_data['model'].values)
test_data['model'] = cls.fit_transform(test_data['model'].values)

train_y = train_data['price']
train_x = train_data.drop(['ID', 'price'], axis = 1)
test_x = test_data.drop('ID', axis = 1)

train_x

,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,2,26,2019,1,3764,4,145,34.5,2.3
1,1,34,2015,1,27056,4,20,60.1,1.0
2,3,9,2019,3,3375,4,145,34.5,2.0
3,3,12,2019,0,5743,0,145,47.9,2.0
4,2,25,2015,1,32782,4,0,65.7,1.0
...,...,...,...,...,...,...,...,...,...
17080,0,0,2018,0,4666,4,145,32.5,3.0
17081,3,10,2020,3,22601,4,145,39.2,2.0
17082,4,93,2018,1,20312,4,145,47.1,1.5
17083,2,26,2018,1,18695,4,145,58.9,1.0


In [4]:
import torch
import torch.optim as optim

torch.manual_seed(1)
x_train = torch.FloatTensor(train_x.values)
x_test = torch.FloatTensor(test_x.values)
y_train = torch.FloatTensor(train_y.values)

y_train

tensor([26299.,  5495., 32400.,  ..., 10950., 13295.,  7299.])

In [5]:
y_train = y_train.view([-1, 1])
y_train

tensor([[26299.],
        [ 5495.],
        [32400.],
        ...,
        [10950.],
        [13295.],
        [ 7299.]])

In [6]:
W = torch.zeros((9, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.00001)

nb_epochs = 1000000

#     # cost 계산

# cost = torch.mean((hypothesis - y_train) ** 2)
# cost = torch.abs(hypothesis - y_train).sum()
# cost

mae = torch.nn.L1Loss()
    
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # Matrix 연산!!
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
   

    cost = mae(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    
    if epoch %10000==0:
        print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()))# 모델 초기화


Epoch    0/1000000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 16819.765625
Epoch 10000/1000000 hypothesis: tensor([17921.6875, 14360.9756, 17972.1191,  ..., 15626.7754, 15836.2754,
        12808.4990]) Cost: 6247.862793
Epoch 20000/1000000 hypothesis: tensor([17934.6934, 14205.9873, 17983.4980,  ..., 15672.7920, 15871.9199,
        12636.8662]) Cost: 6218.888672
Epoch 30000/1000000 hypothesis: tensor([17969.9512, 14053.5879, 18017.7363,  ..., 15719.4668, 15910.8271,
        12455.8047]) Cost: 6192.662109
Epoch 40000/1000000 hypothesis: tensor([18002.4160, 13903.0908, 18049.3828,  ..., 15760.5156, 15945.0020,
        12276.5801]) Cost: 6168.451172
Epoch 50000/1000000 hypothesis: tensor([18029.6758, 13771.3262, 18075.6953,  ..., 15807.2129, 15984.3604,
        12124.8994]) Cost: 6146.175293
Epoch 60000/1000000 hypothesis: tensor([18064.9121, 13650.1865, 18110.1875,  ..., 15857.2451, 16027.9727,
        11982.5928]) Cost: 6125.946289
Epoch 70000/1000000 hypothesis: tensor([1

In [7]:
y_pred = x_test.matmul(W) + b 
y_pred.shape

torch.Size([16831, 1])

In [8]:
y_pred

tensor([[17802.9004],
        [13191.1572],
        [17038.5547],
        ...,
        [13777.0156],
        [17458.4199],
        [10501.2090]], grad_fn=<AddBackward0>)

In [9]:
submit = pd.DataFrame()

submit['ID'] = pd.Series(range(len(y_pred)))
submit['price'] = y_pred.detach().numpy()
submit.to_csv('submit.csv', index = None)